<a href="https://colab.research.google.com/github/Joolyhooly/2021_cphys2/blob/main/FINAL(CNN)_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [108]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

print("Device : ", device) #MACHINE LEARNING에 GPU사용

Device :  cuda


In [109]:
learning_rate = 0.001
batch_size = 256
epochs = 40

In [110]:
# CIFAR10 data 불러오기
train_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [111]:
# CNN Model
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.75
        self.layer1 = nn.Sequential( #nn.Sequential로 아래 코드 한번에 진행되게함
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding='same'), #Input=3, output=32 
            nn.ReLU(),
            nn.BatchNorm2d(32), #batchNorm으로 평균과 분순 학습시 같이 조정
            nn.MaxPool2d(kernel_size=2, stride=2), #Activation map의 특정 영역에서 최댓값 모음.
            nn.Dropout(p=0.2)) #Dropout을 통해 p의 확률로 뉴런을 버려 과적합 방지
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding='same'), #Input=32 output=64
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25))

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding='same'), #input=64 output=128
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.2))
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding='same'), #input= 128, output=64
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25))
        
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding='same'), #input = 64, output = 128
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.5))
        
        
        self.fc1 = nn.Linear(128, 256, bias=True) #input=128 output= 256
        nn.init.xavier_uniform_(self.fc1.weight)
        
        self.fc2 = nn.Linear(256, 10, bias=True) #input = 128 output = 10
        nn.init.xavier_uniform_(self.fc2.weight)
        
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc1(out)
        out = self.fc2(out)
        return out



model = CNN().to(device)


In [112]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #Adam 사용.

In [113]:
#train 함수 정의
def train_loop(dataloader, model, loss_fn, optimizer): 
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader): #batch로 나누어주기

        X = X.to(device)
        y = y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= num_batches
    correct /= size
    print(f"Train - Avg loss: {train_loss:>8f},  Accuracy: {(100*correct):>0.1f}%", end = "  ")

    return train_loss, correct

#test 함수 정의
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test - Avg loss: {test_loss:>8f}  Accuracy: {(100*correct):>0.1f}%")

    return test_loss, correct

In [118]:
train_loss = []
train_acc = []
test_loss = []
test_acc = []
for t in range(epochs): # train loss, test loss for문에 넣음으로써 한번에 볼 수 있음 
    print(f"Epoch {t+1} :", end= "  ") #Epoch 0부터 1 증가
    model.train()
    loss, acc = train_loop(train_dataloader, model, loss_fn, optimizer)
    train_loss.append(loss) #train[]에 loss, acc 값 추가
    train_acc.append(acc)

    loss_t, acc_t = test_loop(test_dataloader, model, loss_fn)
    test_loss.append(loss_t) #test[]에 loss, acc 값 추가
    test_acc.append(acc_t)   

print("Done!")

Epoch 1 :  Train - Avg loss: 0.377625,  Accuracy: 86.9%  Test - Avg loss: 0.770570  Accuracy: 76.7%
Epoch 2 :  Train - Avg loss: 0.381110,  Accuracy: 86.9%  Test - Avg loss: 0.756892  Accuracy: 77.3%
Epoch 3 :  Train - Avg loss: 0.381685,  Accuracy: 86.9%  Test - Avg loss: 0.744309  Accuracy: 77.2%
Epoch 4 :  Train - Avg loss: 0.375269,  Accuracy: 86.9%  Test - Avg loss: 0.766148  Accuracy: 77.8%
Epoch 5 :  Train - Avg loss: 0.372584,  Accuracy: 87.1%  Test - Avg loss: 0.729382  Accuracy: 78.1%
Epoch 6 :  Train - Avg loss: 0.375077,  Accuracy: 87.0%  Test - Avg loss: 0.751266  Accuracy: 77.9%
Epoch 7 :  Train - Avg loss: 0.373522,  Accuracy: 87.1%  Test - Avg loss: 0.745657  Accuracy: 77.7%
Epoch 8 :  Train - Avg loss: 0.373661,  Accuracy: 87.0%  Test - Avg loss: 0.755826  Accuracy: 77.4%
Epoch 9 :  Train - Avg loss: 0.369285,  Accuracy: 87.4%  Test - Avg loss: 0.741098  Accuracy: 77.8%
Epoch 10 :  Train - Avg loss: 0.373686,  Accuracy: 86.9%  Test - Avg loss: 0.771049  Accuracy: 77.4%

In [120]:
print('test accuracy: {}'.format(test_acc[-1]))

test accuracy: 0.7714


# 보고서
Cuda를 활용한 gpu모드로 동작시켜줌으로써 더 빠르게 학습할 수 있도록 하였다.
optimizer의  learning rate를 0.001로, batch_size를 256으로 두어 train_data를 mini_batch화 했고, epoch를 40번으로 여러번 돌려 학습 시켰다.
Train_data = dataset.CIFAR10과 test_data = dataset.CIFAR10을 통해 torchvision에 있는 dataset인 CIFAR10 data를 불러왔다.
CiFAR_10 dataset은 32*32 size의 image를 6만개 포함하고, 3개의 RGB channel이 존재한다. 
즉 60000*32*32*3개가 존재하는 것 이다. 이 중 10000개의 데이터는 test data로 들어가고, 50000개의 데이터는 train data로 들어간다. 
Model은 총 5개의 Conv2d 레이어로 구성하였고, 필터의 사이즈는 kernel size와 같은데, 3*3 크기의 필터를 이용하여 stride의 크기인 
1칸씩 이동시켜가며 진행하였다. 각 conv2d 레이어에서 activation function은 max(0, x)를 의미하는 함수인 ReLU로 
미분 값이 0과 1만을 가지고 계속해서 작아지는 문제를 줄여주었다.
그리고, 각 레이어마다 Batch normalization을 통해 평균과 분산 조정 신경망 안에 넣어 학습 시 같이 조정될 수 있도록 하였고, 
maxpool2d로 activation map에서 특정 영역의 최댓값만 모으는 과정을 처리하였다. 
그 다음엔 dropout에서 p의 확률로 뉴런을 버려 과적합을 방지하는 것을 각 레이어에 적용하였다. 
이 conv2d함수는 2차원의 함수이기 때문에, 1차원으로 내려주는 과정이 필요하다. 이를 flatten이라고 하며, 
Forward에서 view()를 통해 그것을 진행하였고, 이를 softmax 함수를 통해 0~9사이의 값인 총 10개의 클래스로 분류하는 과정을 진행하여 주었다.
이후 model 학습 solution 수렴을 위한 최적화 모델로 다중분류에 사용하는 대표적인 손실함수 
CrossEntropyloss와 gradient를 구해서 weight값을 변화시키는 optimizer로 Adam을 사용했다.
Train함수 정의 후 train 함수의 예측 및 손실을 계산하고, backpropagation를 진행시켜 더 나은 결과값을 얻고자하였다. 
For t in range의 for문에 train loss, test loss를 넣어 한번에 볼 수 있게 했다. 
이후 append를 통해 train[]과 test[]에 loss와 acc값을 각 각 추가해주었고, print(‘test accuracy’)를 통해 test의 정확도를 도출했고, 
그 결과 0.7714의 정확도를 보이는것을 알 수 있었다.